<a href="https://colab.research.google.com/github/oytungunes/ResearchBilkent/blob/main/PeopleWalking_Range_Doppler_and_Spectrogram_BiLSTM_threeclass_noaugment_ipynb_adl%C4%B1_not_defterinin_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Burada people walking verisi ile fw, sw, ve swp ile sadece range-doppler 2D image datası ve spectrogram datası LSTM ile multiclass train edildi.

#Mean test accuracy is 0.558, max test accuracy is 0.573, min test accuracy is 0.538,  std of test accuracy is 0.015.
#Time elapsed through all process: 352.849, sec

In [1]:
from keras.models import Sequential
from tensorflow.keras import layers
from keras.models import Model
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Normalization, Input, Conv2D, MaxPooling2D, Concatenate, GRU, LSTM, GRU, TimeDistributed, Bidirectional
import tensorflow as tf
import scipy.io
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
from numpy.random import seed
from sklearn.model_selection import KFold, StratifiedKFold
import time
from sklearn.metrics import precision_recall_fscore_support
from keras.callbacks import EarlyStopping
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras import backend as K
import gc
import os
from keras.layers import LeakyReLU
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
# Load Range-Doppler fast data
range_doppler_fast = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/range_doppler_fast.mat')
range_doppler_fast = range_doppler_fast['range_doppler_fast']
range_doppler_fast = np.transpose(range_doppler_fast, (2, 0, 1)) # label x width x height
range_doppler_fast_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/range_doppler_fast_label.mat')
range_doppler_fast_label = range_doppler_fast_label['range_doppler_fast_label']

# Load Range-Doppler slow data

range_doppler_slow = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/range_doppler_slow.mat')
range_doppler_slow = range_doppler_slow['range_doppler_slow']
range_doppler_slow = np.transpose(range_doppler_slow, (2, 0, 1))# label x width x height
range_doppler_slow_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/range_doppler_slow_label.mat')
range_doppler_slow_label = range_doppler_slow_label['range_doppler_slow_label']

# Load Range-Doppler slow with pockets data
range_doppler_swp = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/range_doppler_swp.mat')
range_doppler_swp = range_doppler_swp['range_doppler_swp']
range_doppler_swp = np.transpose(range_doppler_swp, (2, 0, 1))# label x width x height
range_doppler_swp_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/range_doppler_swp_label.mat')
range_doppler_swp_label = range_doppler_swp_label['range_doppler_swp_label']


# Load Spectrogram fast data
spectrogram_fast_resized = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/spectrogram_fast_resized.mat')
spectrogram_fast_resized = spectrogram_fast_resized['spectrogram_fast_resized']
spectrogram_fast_resized = np.transpose(spectrogram_fast_resized, (2, 0, 1)) # label x width x height
spectrogram_fast_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/spectrogram_fast_label.mat')
spectrogram_fast_label = spectrogram_fast_label['spectrogram_fast_label']


# Load Spectrogram slow data
spectrogram_slow_resized= scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/spectrogram_slow_resized.mat')
spectrogram_slow_resized = spectrogram_slow_resized['spectrogram_slow_resized']
spectrogram_slow_resized = np.transpose(spectrogram_slow_resized, (2, 0, 1)) # label x width x height
spectrogram_slow_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/spectrogram_slow_label.mat')
spectrogram_slow_label = spectrogram_slow_label['spectrogram_slow_label']

# Load Spectrogram slow with pockets data
spectrogram_swp = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/spectrogram_swp.mat')
spectrogram_swp = spectrogram_swp['spectrogram_swp']
spectrogram_swp = np.transpose(spectrogram_swp, (2, 0, 1))# label x width x height
spectrogram_swp_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/spectrogram_swp_label.mat')
spectrogram_swp_label = spectrogram_swp_label['spectrogram_swp_label']


In [21]:
print(spectrogram_fast_resized.shape)
print(spectrogram_slow_resized.shape)
print(spectrogram_swp.shape)
print(spectrogram_fast_label.shape)
print(spectrogram_slow_label.shape)
print(spectrogram_swp_label.shape)


(57, 100, 100)
(57, 100, 100)
(57, 100, 100)
(57, 1)
(57, 1)
(57, 1)


In [22]:
range_doppler_concat_label = np.concatenate((range_doppler_fast_label,range_doppler_slow_label,range_doppler_swp_label),axis=0)
spectrogram_concat_label = np.concatenate((spectrogram_fast_label,spectrogram_slow_label,spectrogram_swp_label),axis=0)

# One-Hot Encode Label Spectrogram
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(spectrogram_concat_label)
# binary encode
onehot_encoder = OneHotEncoder(sparse_output=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
spectrogram_concat_label_onehotencoded = onehot_encoder.fit_transform(integer_encoded)

# One-Hot Encode Label Range-Doppler
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(range_doppler_concat_label)
# binary encode
onehot_encoder = OneHotEncoder(sparse_output=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
range_doppler_concat_label_onehotencoded = onehot_encoder.fit_transform(integer_encoded)



/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [23]:
# Concat range-doppler data
range_doppler_concat = np.concatenate((range_doppler_fast,range_doppler_slow,range_doppler_swp),axis=0) # fast =fall, slow=walking
range_doppler_concat = range_doppler_concat[:,:,:,np.newaxis]

# Shuffle concat range doppler
shuffle_indx = random.sample(range(0, range_doppler_concat.shape[0]), range_doppler_concat.shape[0]) # split validation data
range_doppler_concat_shuffle = range_doppler_concat[shuffle_indx,:,:,:]
range_doppler_concat_label_shuffle_onehotencoded = range_doppler_concat_label_onehotencoded[shuffle_indx,:]
range_doppler_concat_label_shuffle = range_doppler_concat_label[shuffle_indx,:]



# Concat spectrogram data

spectrogram_concat = np.concatenate((spectrogram_fast_resized,spectrogram_slow_resized,spectrogram_swp),axis=0)
spectrogram_concat = spectrogram_concat[:,:,:,np.newaxis]

# Shuffle concat spectrogram
spectrogram_concat_shuffle = spectrogram_concat[shuffle_indx,:,:,:]
spectrogram_concat_label_shuffle_onehotencoded = spectrogram_concat_label_onehotencoded[shuffle_indx,:]
spectrogram_concat_label_shuffle = spectrogram_concat_label[shuffle_indx,:]

In [24]:
def split_and_augmentation_of_training(spectrogram_concat_shuffle_train,range_doppler_concat_shuffle_train,label_shuffle_train):

  dummy_label = np.zeros((spectrogram_concat_shuffle_train.shape[0],1))
  for randomlist_for_train_indx, randomlist_for_validation_indx in kfold.split(spectrogram_concat_shuffle_train,dummy_label):
    randomlist_for_validation_indx

  # get validation data
  spectrogram_validation_data = spectrogram_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]
  range_doppler_validation_data = range_doppler_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]

  # get training data
  spectrogram_training_data = spectrogram_concat_shuffle_train[randomlist_for_train_indx,:,:,:]
  range_doppler_training_data = range_doppler_concat_shuffle_train[randomlist_for_train_indx,:,:,:]


  #get labels
  validation_labels = label_shuffle_train[randomlist_for_validation_indx,:]
  training_labels = label_shuffle_train[randomlist_for_train_indx,:]


  # get CLASS1 and CLASS2 indexes of training dat

  class1_indexes = np.where(training_labels == 0)[0]
  class2_indexes = np.where(training_labels == 1)[0]
  class3_indexes = np.where(training_labels == 2)[0]


  class1_spectrograms_train = spectrogram_training_data[class1_indexes,:,:,:]
  class2_spectrograms_train = spectrogram_training_data[class2_indexes,:,:,:]
  class3_spectrograms_train = spectrogram_training_data[class3_indexes,:,:,:]


  class1_rangedoppler_train = range_doppler_training_data[class1_indexes,:,:,:]
  class2_rangedoppler_train = range_doppler_training_data[class2_indexes,:,:,:]
  class3_rangedoppler_train = range_doppler_training_data[class3_indexes,:,:,:]


  size_of_samples_class1= class1_spectrograms_train.shape[0]
  size_of_samples_class2 = class2_spectrograms_train.shape[0]
  size_of_samples_class3 = class3_spectrograms_train.shape[0]


  class1_label = np.zeros((size_of_samples_class1,1))
  class2_label = np.ones((size_of_samples_class2,1))
  class3_label = 2*np.ones((size_of_samples_class3,1))


  # unify data
  spectrogram_training_data = np.concatenate((class3_spectrograms_train,class2_spectrograms_train,class1_spectrograms_train),axis=0)
  range_doppler_training_data = np.concatenate((class3_rangedoppler_train,class2_rangedoppler_train,class1_rangedoppler_train),axis=0)


  training_labels = np.concatenate((class3_label,class2_label,class1_label),axis=0)


  return (spectrogram_training_data,range_doppler_training_data,training_labels,\
     spectrogram_validation_data,range_doppler_validation_data, validation_labels)


In [25]:
def normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable):
  # ---------------- Normalize Inputs ----------------
  if normalize_inputs_enable == True:
    layer = Normalization(axis=None)
    layer.adapt(range_doppler_concat_shuffle)
    range_doppler_concat_shuffle = layer(range_doppler_concat_shuffle)
  else:
    range_doppler_concat_shuffle = range_doppler_concat_shuffle
  return(range_doppler_concat_shuffle)


normalize_inputs_enable = 1
range_doppler_concat_shuffle = normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable)
spectrogram_concat_shuffle = normalize_inputs(spectrogram_concat_shuffle, normalize_inputs_enable)
range_doppler_concat_shuffle = np.float32(range_doppler_concat_shuffle)
spectrogram_concat_shuffle = np.float32(spectrogram_concat_shuffle)


In [26]:
#n_features = range_doppler_concat_shuffle.shape[1]
#n_steps = range_doppler_concat_shuffle.shape[2]
range_doppler_concat_shuffle = np.transpose(range_doppler_concat_shuffle, axes = (0,2,1,3))
spectrogram_concat_shuffle = np.transpose(spectrogram_concat_shuffle, axes = (0,2,1,3))

In [27]:
print(range_doppler_concat.shape)
print(range_doppler_concat_label_shuffle.shape)

(171, 100, 100, 1)
(171, 1)


In [30]:
t = time.time()
# ---------- Parameters ----------------
augmentation_enable = True
normalize_inputs_enable = True
num_folds = 5
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state = None) # random_state = 1 ile split run'dan run'a sabit.
test_accuracy_per_run = []
f1_score_per_run = []
epoch_number = 100
batch_size = 32
dense_size = 64
dropout_prob_dense = 0.5
repeat_of_mixup = 5
number_of_repeat = 3
unit_number_of_lstm = 16
dense_unit_of_range_doppler_function = 32
dense_unit_of_spectrogram_function = 8
decoder_dense_unit = 32
for repeat_run_number in range(number_of_repeat):
  test_accuracy_per_fold = []
  f1_score_per_fold = []
  if repeat_run_number > 0:
    del range_doppler_concat_shuffle_test
    del spectrogram_concat_shuffle_test
    del range_doppler_training_data
    del range_doppler_concat_shuffle_train
    del spectrogram_concat_shuffle_train
    del spectrogram_training_data


  #print(range_doppler_concat_label_shuffle_onehotencoded.shape)
  #print(range_doppler_concat_label_shuffle_onehotencoded.shape)

  for randomlist_for_train_indx, randomlist_for_test_indx in kfold.split(range_doppler_concat_shuffle,range_doppler_concat_label_shuffle):
    gc.collect()
    K.clear_session()

    # test data
    range_doppler_concat_shuffle_test = range_doppler_concat_shuffle[randomlist_for_test_indx,:,:,:]
    spectrogram_concat_shuffle_test = spectrogram_concat_shuffle[randomlist_for_test_indx,:,:,:]
    test_label = range_doppler_concat_label_shuffle[randomlist_for_test_indx,:]
    #train data
    range_doppler_concat_shuffle_train = range_doppler_concat_shuffle[randomlist_for_train_indx,:,:,:]
    spectrogram_concat_shuffle_train = spectrogram_concat_shuffle[randomlist_for_train_indx,:,:,:]
    label_shuffle_train = spectrogram_concat_label_shuffle[randomlist_for_train_indx,:]
     # ---------------- Split Data ----------------
    (spectrogram_training_data,range_doppler_training_data,training_labels,\
     spectrogram_validation_data,range_doppler_validation_data, validation_labels)  =\
      split_and_augmentation_of_training(spectrogram_concat_shuffle_train,range_doppler_concat_shuffle_train,label_shuffle_train)


    # One-Hot Encode Label Train
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(training_labels)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse_output=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    training_labels_onehotencoded = onehot_encoder.fit_transform(integer_encoded)

        # One-Hot Encode Label Validation
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(validation_labels)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse_output=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    validation_labels_onehotencoded = onehot_encoder.fit_transform(integer_encoded)


       # One-Hot Encode Label Test
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(test_label)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse_output=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    test_label_onehotencoded = onehot_encoder.fit_transform(integer_encoded)





    # ---------------- Neural Network Architecture ----------------


    def lstm_encoder_network_1(input_shape):
        input = Input(shape=input_shape)
        x = Bidirectional(LSTM(unit_number_of_lstm, return_sequences=True, dropout = 0.5))(input)
        x = Flatten()(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(dense_unit_of_range_doppler_function)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        x = Dropout(dropout_prob_dense)(x)
        return Model(input, x)

    def lstm_encoder_network_2(input_shape):
        input = Input(shape=input_shape)
        x = Bidirectional(LSTM(unit_number_of_lstm, return_sequences=True, dropout = 0.5))(input)
        x = Flatten()(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(dense_unit_of_spectrogram_function)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        x = Dropout(dropout_prob_dense)(x)
        return Model(input, x)

    def decoder_for_concat(input_shape):
      input = Input(shape=input_shape)
      x = Dense(decoder_dense_unit)(input)
      x = BatchNormalization()(x)
      x = LeakyReLU()(x)
      x = Dropout(0.3)(x)
      x = Dense(dense_size)(x)
      x = LeakyReLU()(x)
      x = Dropout(dropout_prob_dense)(x)
      x = Dense(3, activation="softmax")(x)
      return Model(input, x)

    input_shape = range_doppler_concat_shuffle.shape[1:3]
    base_network_lstm = lstm_encoder_network_1(input_shape)
    range_doppler_input  = Input(shape=input_shape)
    processed_range_doppler  = base_network_lstm(range_doppler_input)

    input_shape = spectrogram_concat_shuffle_train.shape[1:3]
    base_network_lstm_2 = lstm_encoder_network_2(input_shape)
    spectrogram_input  = Input(shape=input_shape)
    processed_spectrogram  = base_network_lstm_2(spectrogram_input)

    concat_layer = Concatenate()([processed_range_doppler, processed_spectrogram])

    base_decoder_network = decoder_for_concat((concat_layer.shape[1],))
    out = base_decoder_network(concat_layer)

    model = Model(inputs=[range_doppler_input, spectrogram_input], outputs=[out])
    #if repeat_run_number == 0:
     # print(base_network_lstm.summary())
     # print(base_network_lstm_2.summary())
     # print(base_decoder_network.summary())
    # ---------------- Compile and Fit ----------------
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    earlyStopping = EarlyStopping(monitor='val_loss', patience=15, verbose=0,restore_best_weights=True, mode='min')
    history = model.fit((range_doppler_training_data, spectrogram_training_data),(training_labels_onehotencoded),
                    epochs=epoch_number,
                    batch_size=batch_size,
                    shuffle = True,
                    callbacks=[earlyStopping],
                    validation_data = ((range_doppler_validation_data, spectrogram_validation_data) , (validation_labels_onehotencoded)))
    test_loss, test_accuracy  = model.evaluate([range_doppler_concat_shuffle_test, spectrogram_concat_shuffle_test],\
                                               [test_label_onehotencoded],
                  batch_size=batch_size)
    gc.collect()
    # ---------------- Get Test Results ----------------
    test_accuracy_per_fold.append(test_accuracy)

    del model
  test_accuracy_per_run.append(sum(test_accuracy_per_fold)/num_folds)
  print(test_accuracy_per_run)
print(f'Mean test accuracy is {"{:.3f}".format(sum(test_accuracy_per_run)/number_of_repeat)},\
 max test accuracy is {"{:.3f}".format(max(test_accuracy_per_run))},\
 min test accuracy is {"{:.3f}".format(min(test_accuracy_per_run))}, \
 std of test accuracy is {"{:.3f}".format(np.std(test_accuracy_per_run, axis=0))}.')
elapsed = time.time() - t
print(f'Time elapsed through all process: {"{:.3f}".format(elapsed)}, sec')

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 270ms/step - accuracy: 0.2220 - loss: 1.4875 - val_accuracy: 0.2593 - val_loss: 1.0755
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.3634 - loss: 1.1719 - val_accuracy: 0.2963 - val_loss: 1.0347
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.3519 - loss: 1.2148 - val_accuracy: 0.5556 - val_loss: 0.9830
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.3922 - loss: 1.2847 - val_accuracy: 0.6296 - val_loss: 0.9492
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.3567 - loss: 1.2264 - val_accuracy: 0.6667 - val_loss: 0.9318
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.3891 - loss: 1.2992 - val_accuracy: 0.5556 - val_loss: 0.9308
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.4036 - loss: 1.3210 - val_accuracy: 0.5926 - val_loss: 0.9288
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.4533 - loss: 1.2222 - val_accuracy: 0.5556 - val_loss

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 271ms/step - accuracy: 0.3346 - loss: 1.5133 - val_accuracy: 0.3333 - val_loss: 1.0203
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.2934 - loss: 1.4368 - val_accuracy: 0.4815 - val_loss: 0.9407
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.3965 - loss: 1.2429 - val_accuracy: 0.5556 - val_loss: 0.8905
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.4313 - loss: 1.1840 - val_accuracy: 0.5185 - val_loss: 0.8595
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.4553 - loss: 1.0784 - val_accuracy: 0.5185 - val_loss: 0.8462
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.4131 - loss: 1.3456 - val_accuracy: 0.4815 - val_loss: 0.8346
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.3975 - loss: 1.1676 - val_accuracy: 0.4815 - val_loss: 0.8225
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.5000 - loss: 1.0144 - val_accuracy: 0.4815 - val_loss

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 263ms/step - accuracy: 0.2460 - loss: 1.6619 - val_accuracy: 0.2593 - val_loss: 1.1310
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.3814 - loss: 1.2739 - val_accuracy: 0.5185 - val_loss: 1.0835
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.3657 - loss: 1.2931 - val_accuracy: 0.5185 - val_loss: 1.0612
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.3590 - loss: 1.2876 - val_accuracy: 0.5185 - val_loss: 1.0480
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.3897 - loss: 1.1665 - val_accuracy: 0.5185 - val_loss: 1.0474
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.4235 - loss: 1.1584 - val_accuracy: 0.5185 - val_loss: 1.0409
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.2627 - loss: 1.3917 - val_accuracy: 0.5185 - val_loss: 1.0312
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.4448 - loss: 1.1582 - val_accuracy: 0.5185 - val_loss

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 265ms/step - accuracy: 0.2580 - loss: 1.4227 - val_accuracy: 0.3704 - val_loss: 1.0599
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.3199 - loss: 1.2157 - val_accuracy: 0.4444 - val_loss: 1.0302
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.4053 - loss: 1.2213 - val_accuracy: 0.5926 - val_loss: 0.9938
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.4048 - loss: 1.1854 - val_accuracy: 0.6667 - val_loss: 0.9657
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.2841 - loss: 1.3879 - val_accuracy: 0.6667 - val_loss: 0.9547
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.3439 - loss: 1.3097 - val_accuracy: 0.5556 - val_loss: 0.9490
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.3491 - loss: 1.2502 - val_accuracy: 0.5556 - val_loss: 0.9427
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.3273 - loss: 1.1427 - val_accuracy: 0.5556 - val_loss

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 263ms/step - accuracy: 0.3684 - loss: 1.4008 - val_accuracy: 0.5926 - val_loss: 1.0274
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.3402 - loss: 1.3497 - val_accuracy: 0.5556 - val_loss: 1.0083
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.3913 - loss: 1.3121 - val_accuracy: 0.7037 - val_loss: 0.9973
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.4225 - loss: 1.1322 - val_accuracy: 0.6667 - val_loss: 0.9844
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.4038 - loss: 1.2549 - val_accuracy: 0.6667 - val_loss: 0.9681
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.3918 - loss: 1.1579 - val_accuracy: 0.6296 - val_loss: 0.9535
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.3439 - loss: 1.2364 - val_accuracy: 0.5556 - val_loss: 0.9506
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.3772 - loss: 1.1471 - val_accuracy: 0.5556 - val_loss

In [31]:
print(f'Mean test accuracy is {"{:.3f}".format(sum(test_accuracy_per_run)/number_of_repeat)},\
 max test accuracy is {"{:.3f}".format(max(test_accuracy_per_run))},\
 min test accuracy is {"{:.3f}".format(min(test_accuracy_per_run))}, \
 std of test accuracy is {"{:.3f}".format(np.std(test_accuracy_per_run, axis=0))}.')
elapsed = time.time() - t
print(f'Time elapsed through all process: {"{:.3f}".format(elapsed)}, sec')

Mean test accuracy is 0.558, max test accuracy is 0.573, min test accuracy is 0.538,  std of test accuracy is 0.015.
Time elapsed through all process: 370.186, sec
